In [11]:
import pickle
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from lightgbm import LGBMClassifier, early_stopping

In [12]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.head()

,clean_comment,category
0,"film absolutely awful, but nevertheless, hilar...",0
1,well since seeing part 1 3 honestly say never ...,0
2,got see film preview dazzled it. not typical r...,1
3,adaptation positively butcher classic beloved ...,0
4,rzone awful movie! simple. seems tried make mo...,0


In [13]:
df = df.dropna(subset=['clean_comment', 'category'])

# Train/Test split (stratified)
X_train_text, X_test_text, y_train, y_test = train_test_split(
    df['clean_comment'],
    df['category'],
    test_size=0.2,
    random_state=42,
    stratify=df['category']
)

In [14]:
# TF-IDF 
vectorizer = TfidfVectorizer(
    ngram_range=(1, 3),
    max_features=10000,
    lowercase=True,
    strip_accents='unicode'
)
X_train_vec = vectorizer.fit_transform(X_train_text)
X_test_vec  = vectorizer.transform(X_test_text)

In [15]:
# Imbalance handling via weighting 
# scale_pos_weight = (#negative / #positive) on TRAIN set only
pos = (y_train == 1).sum()
neg = (y_train == 0).sum()
scale_pos_weight = (neg / max(pos, 1))

In [16]:
# 4) Optuna objective (optimize for Accuracy on test set)
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1200),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.2, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 16, 256),
        "max_depth": trial.suggest_int("max_depth", -1, 32),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "min_split_gain": trial.suggest_float("min_split_gain", 0.0, 1.0),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "subsample_freq": trial.suggest_int("subsample_freq", 0, 10),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 5.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 5.0),
        "random_state": 42,
        "n_jobs": -1,
        "class_weight": "balanced",
        "objective": "binary",
        "metric": "binary_logloss",
        "verbose": -1
    }

    model = LGBMClassifier(**params)

    # early_stopping() callback instead of early_stopping_rounds arg
    model.fit(
        X_train_vec, y_train,
        eval_set=[(X_test_vec, y_test)],
        eval_metric="binary_logloss",
        callbacks=[early_stopping(stopping_rounds=100)]
    )

    y_pred = model.predict(X_test_vec)
    acc = accuracy_score(y_test, y_pred)
    return acc

# Run Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=40, show_progress_bar=False)

print("Best trial:", study.best_trial.number)
print("Best value (Accuracy):", study.best_value)
print("Best params:", study.best_params)

[I 2025-08-14 15:20:39,430] A new study created in memory with name: no-name-a435da8a-03a4-4fd5-b73d-c54b2630588c


Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[842]	valid_0's binary_logloss: 0.340198


C:\jupyter\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-08-14 15:22:15,506] Trial 0 finished with value: 0.8562537643043565 and parameters: {'n_estimators': 842, 'learning_rate': 0.012976483348855468, 'num_leaves': 178, 'max_depth': 19, 'min_child_samples': 92, 'min_split_gain': 0.3135781464219839, 'subsample': 0.6421987485366988, 'subsample_freq': 3, 'colsample_bytree': 0.8131390231861435, 'reg_alpha': 4.866504919191747, 'reg_lambda': 0.3400571935390123}. Best is trial 0 with value: 0.8562537643043565.


Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[370]	valid_0's binary_logloss: 0.295631


C:\jupyter\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-08-14 15:23:58,625] Trial 1 finished with value: 0.8703071672354948 and parameters: {'n_estimators': 371, 'learning_rate': 0.061079869969120536, 'num_leaves': 69, 'max_depth': 29, 'min_child_samples': 31, 'min_split_gain': 0.15262424078030123, 'subsample': 0.9316017363486365, 'subsample_freq': 0, 'colsample_bytree': 0.9485204768457853, 'reg_alpha': 1.6802995511936496, 'reg_lambda': 4.756802518496831}. Best is trial 1 with value: 0.8703071672354948.


...

Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[1012]	valid_0's binary_logloss: 0.345592


C:\jupyter\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-08-14 16:10:33,197] Trial 27 finished with value: 0.8526400321220639 and parameters: {'n_estimators': 1012, 'learning_rate': 0.06353690613641673, 'num_leaves': 82, 'max_depth': 2, 'min_child_samples': 19, 'min_split_gain': 0.9246555108623714, 'subsample': 0.85408818391377, 'subsample_freq': 9, 'colsample_bytree': 0.6396360716485734, 'reg_alpha': 1.4343137423825258, 'reg_lambda': 3.014533175361994}. Best is trial 21 with value: 0.8767315800040153.


Training until validation scores don't improve for 100 rounds


[W 2025-08-14 16:12:35,142] Trial 28 failed with parameters: {'n_estimators': 1068, 'learning_rate': 0.010403168924217247, 'num_leaves': 37, 'max_depth': 9, 'min_child_samples': 30, 'min_split_gain': 0.40560450384897834, 'subsample': 0.6729542736802457, 'subsample_freq': 7, 'colsample_bytree': 0.6873427307866279, 'reg_alpha': 2.419226401226351, 'reg_lambda': 4.207301559204048} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\jupyter\venv\Lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\latif\AppData\Local\Temp\ipykernel_9112\2468078835.py", line 26, in objective
    model.fit(
    ~~~~~~~~~^
        X_train_vec, y_train,
        ^^^^^^^^^^^^^^^^^^^^^
    ...<2 lines>...
        callbacks=[early_stopping(stopping_rounds=100)]
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "C:\jupyter\venv\Lib\site-packages\lightgbm\sklearn.py", line 1560, 

KeyboardInterrupt: 

In [17]:
best_params = study.best_params.copy()
final_model = LGBMClassifier(
    **best_params,
    random_state=42,
    n_jobs=-1,
    class_weight="balanced",
    objective="binary",
    metric="binary_logloss",
    verbose=-1,
    # LightGBM-এ scale_pos_weight অতিরিক্ত bias যোগ করে; চাইলে টিউনও করা যায়।
    # এখানে আমরা ইমব্যাল্যান্স থাকলে ব্যবহার করছি:
    scale_pos_weight=scale_pos_weight if scale_pos_weight > 1 else 1.0
)

final_model.fit(
    X_train_vec, y_train,
    eval_set=[(X_test_vec, y_test)],
    eval_metric="binary_logloss",
    callbacks=[],
    early_stopping_rounds=100
)

TypeError: LGBMClassifier.fit() got an unexpected keyword argument 'early_stopping_rounds'